In [1]:
import fire
from starfish.spots import DecodeSpots
from starfish.core.imagestack.imagestack import ImageStack
from starfish.types import Axes, FunctionSource
from starfish.spots import FindSpots, DecodeSpots, AssignTargets
from starfish.core.types import SpotFindingResults
from starfish.core.intensity_table.decoded_intensity_table import DecodedIntensityTable
from starfish import Experiment, FieldOfView
from starfish.image import LearnTransform, ApplyTransform, Filter, Segment

#### Auxiliary functions:
def register(
    image_stack: ImageStack,
    reference_stack: ImageStack,
) -> ImageStack:
    learn_translation = LearnTransform.Translation(
        reference_stack=reference_stack, axes=Axes.ROUND, upsampling=100)

    transforms_list = learn_translation.run(
        image_stack.reduce({Axes.CH, Axes.ZPLANE}, func='max'))

    warp = ApplyTransform.Warp()
    registered = warp.run(image_stack, transforms_list=transforms_list,
                            in_place=False, verbose=False)

    return registered

def find_spots(
    image_stack: ImageStack, 
    reference_stack: ImageStack
) -> SpotFindingResults:
    """Detect spots using laplacian of gaussians approach."""
    bd = FindSpots.BlobDetector(
                min_sigma=1,
                max_sigma=2,
                num_sigma=30,
                threshold=0.08,
                is_volume=False,
                measurement_type='mean')
    dots_max = reference_stack.reduce((Axes.ROUND, Axes.ZPLANE),
                                       func='max')
    # Locate spots in reference image
    spots = bd.run(image_stack=image_stack, reference_image=dots_max)

    return spots

def decode(spots: SpotFindingResults, experiment) -> DecodedIntensityTable:
    """Decode pixel traces using the codebook."""
    decoder = DecodeSpots.PerRoundMaxChannel(
        codebook=experiment.codebook,
    )
    decoded = decoder.run(spots=spots)

    return decoded

"""
def decode_starfish(spots: SpotFindingResults, json_path) -> DecodedIntensityTable:

    exp = Experiment.from_json(json_path)
    codebook = exp.codebook
    decoder = DecodeSpots.PerRoundMaxChannel(
        codebook=codebook,
    )
    decoded_spots = {}
    for fov_name, spot in spots.items():
        decoded = decoder.run(spots=spot)
        decoded_spots[fov_name] = decoded

    return decoded_spots
"""

def process_fov(
    fov_name: str
):
    exp = Experiment.from_json('experiment.json')
    fov = exp[fov_name]
    primary = fov.get_image(FieldOfView.PRIMARY_IMAGES)
    reference = fov.get_image('anchor_dots')
    dapi_rounds = fov.get_image('nuclei')
    dapi_ref = fov.get_image('anchor_nuclei')

    #if normalize:
    #    primary = _normalize(image_stack=primary)
            
    #if True:
        #if reg_withDapi:
        #primary_registered = self._register_new(
        #                                 fov_name,
        #                                 image_stack=primary,
        #                                 dapi_ref=dapi_ref,
        #                                 dapi_round=dapi_rounds
        #                                 )
        #    else:
    #primary_registered = register(
    #                        reference_stack=reference,
    #                        image_stack=primary
    #                        )

    filtered_ref = reference
    filtered_imgs = primary#_registered

    spots = find_spots(image_stack=filtered_imgs,
                       reference_stack=filtered_ref)

    decoded = decode(spots, exp)

    return decoded, spots, fov_name


In [2]:
d, s, f = process_fov('fov_001')

100%|██████████| 16/16 [00:00<00:00, 360.94it/s]


In [3]:
d

<xarray.DecodedIntensityTable (features: 160, r: 4, c: 4)>
array([[[2.40619317e-01, 2.21492901e-01, 2.49898598e-01, 4.63263929e-01],
        [2.84320503e-01, 4.01293606e-01, 3.26562047e-01, 3.06552768e-01],
        [3.13283473e-01, 4.24024045e-01, 3.29883456e-01, 3.66445750e-01],
        [4.37971711e-01, 4.89572406e-01, 3.20371181e-01, 4.20684040e-01]],

       [[2.37069979e-01, 2.58681685e-01, 2.07191348e-01, 2.01299027e-01],
        [2.09695369e-01, 1.71287566e-01, 1.57881156e-01, 1.65892988e-01],
        [2.47408599e-01, 2.00332224e-01, 1.47949129e-01, 2.16229200e-01],
        [3.25825125e-01, 2.00965971e-01, 1.47361264e-01, 1.79399490e-01]],

       [[2.25268260e-01, 1.82452321e-01, 1.98201522e-01, 2.88970798e-01],
        [2.22967178e-01, 2.09063888e-01, 2.41924226e-01, 1.89201191e-01],
        [2.59673953e-01, 2.08172828e-01, 1.52169049e-01, 1.79958001e-01],
        [3.24253231e-01, 2.09797204e-01, 1.50278270e-01, 1.82987511e-01]],

       ...,

       [[2.19509166e-05, 2.16077769e-05, 2.06654877e-05, 2.20544589e-05],
        [7.75569454e-02, 8.10161382e-02, 9.24859866e-02, 8.83594751e-02],
        [2.58182287e-01, 2.70507157e-01, 2.20509067e-01, 2.56292522e-01],
        [1.24926649e-01, 9.71849859e-02, 7.43351132e-02, 8.60272795e-02]],

       [[2.70464734e-05, 2.23016687e-05, 2.66391307e-05, 2.36710166e-05],
        [9.05898809e-02, 1.15070596e-01, 1.05118766e-01, 1.05913356e-01],
        [3.15566927e-01, 4.42708760e-01, 3.80234241e-01, 3.21722597e-01],
        [1.46907836e-01, 1.55346513e-01, 1.51159450e-01, 1.31429747e-01]],

       [[4.74397821e-05, 3.98036354e-05, 6.12586009e-05, 4.74288281e-05],
        [1.18953399e-01, 2.11204782e-01, 1.85077369e-01, 1.95950240e-01],
        [5.04259944e-01, 9.97153401e-01, 9.19997871e-01, 7.04963267e-01],
        [1.93085551e-01, 2.96661973e-01, 2.45858252e-01, 2.28167370e-01]]])
Coordinates:
    intensity          (features) float64 0.2406 0.2371 ... 2.705e-05 4.744e-05
    z                  (features) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    y                  (features) int64 197 196 195 192 192 191 ... 4 4 4 3 3 3
    x                  (features) int64 28 147 154 175 23 ... 51 33 134 74 11
    radius             (features) float64 2.0 2.0 2.0 3.0 ... 2.0 2.0 2.0 2.0
    spot_id            (features) int64 0 1 2 3 4 5 ... 154 155 156 157 158 159
    z_min              (features) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    z_max              (features) int64 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    y_min              (features) int64 196 195 194 190 191 189 ... 3 3 3 2 2 2
    y_max              (features) int64 199 198 197 195 194 194 ... 6 6 6 5 5 5
    x_min              (features) int64 27 146 153 173 22 ... 50 32 133 73 10
    x_max              (features) int64 30 149 156 178 25 ... 53 35 136 76 13
  * features           (features) int64 0 1 2 3 4 5 ... 154 155 156 157 158 159
  * r                  (r) int64 0 1 2 3
  * c                  (c) int64 0 1 2 3
    xc                 (features) float64 228.1 347.7 354.8 ... 274.4 211.1
    yc                 (features) float64 198.0 197.0 196.0 ... 3.015 3.015
    zc                 (features) float64 5e-05 5e-05 5e-05 ... 5e-05 5e-05
    target             (features) <U5 'nan' 'nan' 'nan' ... 'ST-3' 'nan' 'nan'
    distance           (features) float64 0.6781 0.6825 0.6746 ... 0.7209 0.6909
    passes_thresholds  (features) bool False False False ... True False False
Attributes:
    starfish:  {"log": [{"method": "BlobDetector", "arguments": {"min_sigma":...

In [4]:
s

In [7]:
s.data

AttributeError: 'SpotFindingResults' object has no attribute 'data'

In [5]:
dir(d)

['T',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__dask_graph__',
 '__dask_keys__',
 '__dask_layers__',
 '__dask_optimize__',
 '__dask_postcompute__',
 '__dask_postpersist__',
 '__dask_scheduler__',
 '__dask_tokenize__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__div__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '_

In [6]:
d.data

array([[[2.40619317e-01, 2.21492901e-01, 2.49898598e-01, 4.63263929e-01],
        [2.84320503e-01, 4.01293606e-01, 3.26562047e-01, 3.06552768e-01],
        [3.13283473e-01, 4.24024045e-01, 3.29883456e-01, 3.66445750e-01],
        [4.37971711e-01, 4.89572406e-01, 3.20371181e-01, 4.20684040e-01]],

       [[2.37069979e-01, 2.58681685e-01, 2.07191348e-01, 2.01299027e-01],
        [2.09695369e-01, 1.71287566e-01, 1.57881156e-01, 1.65892988e-01],
        [2.47408599e-01, 2.00332224e-01, 1.47949129e-01, 2.16229200e-01],
        [3.25825125e-01, 2.00965971e-01, 1.47361264e-01, 1.79399490e-01]],

       [[2.25268260e-01, 1.82452321e-01, 1.98201522e-01, 2.88970798e-01],
        [2.22967178e-01, 2.09063888e-01, 2.41924226e-01, 1.89201191e-01],
        [2.59673953e-01, 2.08172828e-01, 1.52169049e-01, 1.79958001e-01],
        [3.24253231e-01, 2.09797204e-01, 1.50278270e-01, 1.82987511e-01]],

       ...,

       [[2.19509166e-05, 2.16077769e-05, 2.06654877e-05, 2.20544589e-05],
        [7.75569454